In [ ]:



import pandas as pd
from sqlalchemy import create_engine

import sys
sys.path.append('../')
from src.user_behavior import aggregate_user_behavior
from src.preprocessing import replace_missing
from src.telecom_analysis import identify_top_handsets, identify_top_manufacturers, identify_top_handsets_per_manufacturer, generate_recommendations






In [ ]:
database_name = 'telecom'
table_name= 'xdr_data'

connection_params = { "host": "localhost", "user": "postgres", "password": "00000000",
                    "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data '

df = pd.read_sql(sql_query, con= engine)
df

In [ ]:
database_name = 'processed_telecom'
table_name= 'xdr_data'

connection_params = { "host": "localhost", "user": "postgres", "password": "00000000",
                    "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data '

df = pd.read_sql(sql_query, con= engine)
df

In [ ]:
# Preprocess the dataset
df = replace_missing(df)
df

In [5]:
# Identify the top 10 handsets
top_handsets = identify_top_handsets(df)
print("Top 10 Handsets:")
print(top_handsets)

Top 10 Handsets:
Handset Type
Huawei B528S-23A                19752
Apple iPhone 6S (A1688)          9419
Apple iPhone 6 (A1586)           9023
undefined                        8987
Apple iPhone 7 (A1778)           6326
Apple iPhone Se (A1723)          5187
Apple iPhone 8 (A1905)           4993
Apple iPhone Xr (A2105)          4568
Samsung Galaxy S8 (Sm-G950F)     4520
Apple iPhone X (A1901)           3813
Name: count, dtype: int64


In [ ]:
# Generate recommendations to marketing teams
recommendations = generate_recommendations()
print("\nRecommendations to Marketing Teams:")
print(recommendations)

In [ ]:
# Identify the top 3 handset manufacturers
top_manufacturers = identify_top_manufacturers(df)
print("\nTop 3 Handset Manufacturers:")
print(top_manufacturers)


In [ ]:
# Identify the top 5 handsets per top 3 handset manufacturer
top_handsets_per_manufacturer = identify_top_handsets_per_manufacturer(df)
print("\nTop 5 Handsets per Top 3 Handset Manufacturers:")
print(top_handsets_per_manufacturer)